In [7]:
import bw2io as bi
import bw2data as bd
import bw2calc as bc

In [17]:
bd.projects.migrate_project_25()

Updating all LCIA methods


1it [00:00, 24.77it/s]


Updating all LCI databases


1it [00:00, 16.85it/s]


In [18]:
bd.projects.set_current('as23_project')

In [28]:
del bd.databases['wind-example']

In [29]:
bd.databases

Databases dictionary with 0 objects

In [30]:
PRODUCTION_VOLUME = 1000 # Production volume of wind turbine over lifetime

bd.Database('wind-example').write({
    ('wind-example', "CO2"): {
        "type": "emission",
        "name": "carbon dioxide",
        "unit": "kilogram",
    },
    ('wind-example', "coal"): {
        "type": "emission",
        "name": "coal",
        "unit": "kilogram",
    },
    ('wind-example', 'electricity-mix'): {
        'name': 'Electricity mix',
        'unit': 'kilowatt hour',
        'exchanges': [
            {
                'input': ('wind-example', 'electricity-production-wind'),
                'amount': 0.5,
                'type': 'technosphere',
            },
            {
                'input': ('wind-example', 'electricity-production-coal'),
                'amount': 0.5,
                'type': 'technosphere',
            },
        ]
    },
    ('wind-example', 'electricity-production-coal'): {
        'name': 'Electricity production, coal',
        'unit': 'kilowatt hour',
        'exchanges': [
            {
                'input': ('wind-example', 'coal'),
                'amount': 1,
                'type': 'biosphere',
            },
            {
                'input': ('wind-example', 'CO2'),
                'amount': 1,
                'type': 'biosphere',
            },
        ]
    },
    ('wind-example', 'electricity-production-wind'): {
        'name': 'Electricity production, wind',
        'unit': 'kilowatt hour',
        'exchanges': [
            {
                'input': ('wind-example', 'electricity-production-wind'),
                'amount': 1,
                'type': 'production',
            },
            {
                'input': ('wind-example', 'wind-turbine-construction'),
                'amount': 1/PRODUCTION_VOLUME,
                'type': 'technosphere',
            },
            # maybe add maintenance, oil changes, ...
        ]
    },
    ('wind-example', 'wind-turbine-construction'): {
        'name': 'Wind turbine construction',
        'unit': 'unit',
        'exchanges': [
            {
                'input': ('wind-example', 'wind-turbine-construction'),
                'amount': 1,
                'type': 'production',
            },
            {
                'input': ('wind-example', 'electricity-mix'),
                'amount': 1,
                'type': 'technosphere',
            },
            {
                'input': ('wind-example', 'eol-wind'),
                'amount': 1,
                'type': 'technosphere',
            },
            # aggregate the rest to direct co2 emissions
            {
                'input': ('wind-example', 'CO2'),
                'amount': 1,
                'type': 'biosphere',
            }
        ]
    },
    ('wind-example', 'eol-wind'): {
        'name': "End-of-life, wind turbine",
        'unit': 'unit',
        'exchanges': [
            {
                'input': ('wind-example', 'eol-wind'),
                'amount': 1,
                'type': 'production',
            },
            # aggregate the rest to direct co2 emissions
            {
                'input': ('wind-example', 'CO2'),
                'amount': 1,
                'type': 'biosphere',
            }
        ]
    },
})

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 7/7 [00:00<?, ?it/s]

Vacuuming database 


In [31]:
bd.Method(("GWP", "wind-example")).write([
    (('wind-example', "CO2"), 1),
])

In [32]:
lca = bc.LCA({('wind-example', 'electricity-mix'): 1}, ("GWP", "wind-example"))
lca.lci()
lca.lcia()
lca.score

0.5012506253720875